<a href="https://colab.research.google.com/github/alex-pakalniskis/ua-gis-day-2019/blob/master/notebooks/ua_gis_day_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install open-source GIS software on Google cloud computer

In [0]:
# Install curl (https://curl.haxx.se/), g++ (https://en.wikipedia.org/wiki/GNU_Compiler_Collection), and make (https://en.wikipedia.org/wiki/Make_(software)) on the remote machine you are using
!apt-get install -qq curl g++ make
# Use curl to download zippied spatial indexing software from Open Source Geospatial Foundation (https://www.osgeo.org/) and use tar to unzip
!curl -s -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
# Python library for operating system specific tasks such as changing directories (https://docs.python.org/3/library/os.html)
import os
# Change current working directory to newly unzipped OSGEO spatial indexing software
os.chdir('spatialindex-src-1.8.5')
# Configure software and dependencies for install on remote Google computer
!./configure
# Build software from its Makefile (https://en.wikipedia.org/wiki/Makefile) by invoking make (https://en.wikipedia.org/wiki/Make_(software))
!make
# Copy built software and files to correct locations for accessing later
!make install
# Use pip Python package manager (https://en.wikipedia.org/wiki/Pip_(package_manager)) to install rtree (http://toblerity.org/rtree/)
# Python-wrapper for libspatialindex, a C++ library (https://libspatialindex.org/) for implementing R-tree data access (https://en.wikipedia.org/wiki/R-tree)
# http://toblerity.org/rtree/
!pip -q install rtree
# Configure (symbolic) links with ldconfig (https://linux.die.net/man/8/ldconfig)
!ldconfig
# Python spatial indexing library
import rtree
from rtree import index
from rtree.index import Rtree
# Check for packages which need updating and install the development branch of libspatialindex onto the remote Google cloud computer with a linux OS
!sudo apt-get update && apt-get install -y libspatialindex-dev
# Install descartes (https://pypi.org/project/descartes/) to enable plotting planar geometric vector objects in matplotlib
!pip -q install descartes
# Spatial version of Pandas library for GIS data management and basic analyses (http://geopandas.org/)
!pip -q install geopandas 

# Library for zonal statistics and interpolated point queries (https://pythonhosted.org/rasterstats/)
!pip -q install rasterstats

# Rasterio reads and writes TIFF files and provides a Python API based on Numpy N-dimensional arrays and GeoJSON
!pip -q install rasterio

# Helper libraries
!pip install adjustText
!pip install urbanaccess pandana

# Download files from Google Drive more easily
!pip install gdown

# Upgrade all the installed software
!sudo apt-get upgrade

# Import open-source GIS software libraries for analysis and visualization

In [0]:
# Set the current working directory to "home"
import os
os.chdir("/home/")

# Data management and analysis library with emphasis on tabular data. Widely used in research and industry, as it was initially developed for financial analyses of stocks. Comes preloaded in Colaboratory. 
import pandas as pd

import fiona

import geopandas as gpd

# Colaboratory helper library for importing and downloading locally stored files into the remote Google machine
import google.colab.files

# Library for reading and writing geospatial raster data such as TIFFS (https://rasterio.readthedocs.io/en/stable/quickstart.html)
import rasterio
from rasterio.plot import show

# Library for zonal statistics
import rasterstats
from rasterstats import zonal_stats

# Pyplot sublibrary of matplotlib for legendandary visualization parameterization capabilities
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Great, newer visualization library built-on matplotlib but with more modern styling. Comes preloaded in Colaboratory. (https://seaborn.pydata.org/)
import seaborn as sns

# Styling library to reduce label overlap in matplotlib
from adjustText import adjust_text

# Setting global plot style aesthetics through seaborn
sns.set(context="notebook", palette="colorblind")

# Download US county boundaries shapefile from the US Census with `wget`

In [0]:
!wget -q https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip 

# Unzip the data with `unzip`

In [0]:
!unzip cb_2018_us_county_500k.zip

# Read shapefile data with `geopandas`

In [0]:
county_gdf = gpd.read_file("cb_2018_us_county_500k.shp")

# Check the shapefile [projection](https://en.wikipedia.org/wiki/Map_projection)

In [0]:
county_gdf.crs

# Display the first five rows

In [0]:
county_gdf.head()

# Return the shape of the data set

In [0]:
county_gdf.shape

# Return the unique STATEFP values ([Federal Information Processing Standard state code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code))

In [0]:
county_gdf["STATEFP"].unique()

# Count the number of unique STATEFP codes. What number do you expect?

In [0]:
len(county_gdf["STATEFP"].unique())

# Were you right? If not, what might have you forgotten?

# Subset the country-wide county data set for Arizona. Do the county names look familiar?

In [0]:
county_gdf[county_gdf["STATEFP"] == "04"]

# Plot the subset to visually verify

In [0]:
county_gdf[county_gdf["STATEFP"] == "04"].plot(cmap="Oranges")

# Create a new DataFrame from a copy of the Arizona subset

In [0]:
arizona_counties = county_gdf[county_gdf["STATEFP"] == "04"].copy()

In [0]:
arizona_counties.centroid

In [0]:
fig, ax = plt.subplots(figsize=(10,10))
arizona_counties.plot(ax=ax, cmap="Oranges")
arizona_counties.centroid.plot(ax=ax, cmap="Greens_r")

# Calculate the area of each county
* Project the UTM 12N
* Calculate area
* Convert area from ${m^2}$ to ${km^2}$

In [0]:
arizona_counties.to_crs("EPSG:32612").area*(10**-6)

# Return the outer boundary of counties (i.e. Arizona border)
Likewise, plot the centroid for the state.

In [0]:
fig, ax = plt.subplots()
gpd.GeoSeries(arizona_counties.unary_union).plot(ax=ax, color="orange")
gpd.GeoSeries(arizona_counties.unary_union).centroid.plot(ax=ax, color="green")
plt.show()

# Save the clipped Arizona county boundaries to file

In [0]:
arizona_counties.to_file("arizona_county_boundaries.geojson", driver="GeoJSON")

# Download the Arizona Land Cover change data (stored as a zipped TIFF on Google Drive)

https://drive.google.com/file/d/1bFudz1SDazEHa4liUnA8Xf0uWHyRzA3n/view?usp=sharing

# Slightly tweak the URL to be appropriate for `gdown` usage

https://drive.google.com/uc?id= [FILE ID], where [FILE ID] = 1bFudz1SDazEHa4liUnA8Xf0uWHyRzA3n

In [0]:
!gdown https://drive.google.com/uc?id=1bFudz1SDazEHa4liUnA8Xf0uWHyRzA3n

# Unzip the raster data

In [0]:
!unzip arizona_esa_landcover_1992_2015.zip

# Use `rasterio` to read the data

In [0]:
src = rasterio.open("arizona_esa_landcover_1992_2015.tif")

# Write a convenience function for mapping a band of raster data (with automatic descriptive titles)

In [0]:
def plot_a_band(file, band_number, cmap):

  fig, ax = plt.subplots(figsize=(5,5))

  src = rasterio.open(file)

  show((src, band_number), cmap=cmap, ax=ax)
  plt.title("Land Cover in {}".format(band_number+1991), fontsize="x-large")
  ax.grid("") 
  plt.show()

In [0]:
# Unlike the rest of Python, rasterio is "1"-indexed
plot_a_band("arizona_esa_landcover_1992_2015.tif", 1, "gist_earth_r")

In [0]:
plot_a_band("arizona_esa_landcover_1992_2015.tif", 24, "cividis")

# Use zonal statistics to model land cover change through time

In [0]:
def generate_esa_landcover_timeseries_zonal_statistics(regional_landcover_data,
                                                        zones_statistics_boundaries, 
                                                        identifier_column):
    
    # ESA Land cover categories
    category_map = {0:"No data",
                    10:"Cropland, rainfed",
                    11:"Herbaceous cover",
                    12:"Tree or shrub cover",
                    20:"Cropland, irrigated or post-flooding",
                    30:"Mosaic cropland (>50%) / natural vegetation (tree, shrub, herbaceous cover) (<50%)",
                    40:"Mosaic natural vegetation (tree, shrub, herbaceous cover) (>50%) / cropland (<50%)",
                    50:"Tree cover, broadleaved, evergreen, closed to open (>15%)",
                    60:"Tree cover, broadleaved, deciduous, closed to open (>15%)",
                    61:"Tree cover, broadleaved, deciduous, closed (>40%)",
                    62:"Tree cover, broadleaved, deciduous, open (15-40%)",
                    70:"Tree cover, needleleaved, evergreen, closed to open (>15%)",
                    71:"Tree cover, needleleaved, evergreen, closed (>40%)",
                    72:"Tree cover, needleleaved, evergreen, open (15-40%)",
                    80:"Tree cover, needleleaved, deciduous, closed to open (>15%)",
                    81:"Tree cover, needleleaved, deciduous, closed (>40%)",
                    82:"Tree cover, needleleaved, deciduous, open (15-40%)",
                    90:"Tree cover, mixed leaf type (broadleaved and needleleaved)",
                    100:"Mosaic tree and shrub (>50%) / herbaceous cover (<50%)",
                    110:"Mosaic herbaceous cover (>50%) / tree and shrub (<50%)",
                    120:"Shrubland",
                    121:"Shrubland evergreen",
                    122:"Shrubland deciduous",
                    130:"Grassland",
                    140:"Lichens and mosses",
                    150:"Sparse vegetation (tree, shrub, herbaceous cover) (<15%)",
                    151:"Sparse tree (<15%)",
                    152:"Sparse shrub (<15%)",
                    153:"Sparse herbaceous cover (<15%)",
                    160:"Tree cover, flooded, fresh or brakish water",
                    170:"Tree cover, flooded, saline water",
                    180:"Shrub or herbaceous cover, flooded, fresh/saline/brakish water",
                    190:"Urban areas",
                    200:"Bare areas",
                    201:"Consolidated bare areas",
                    202:"Unconsolidated bare areas",
                    210:"Water bodies",
                    220:"Permanent snow and ice"}
    
    # List of land cover category values from dictionary
    lulcc_list = list(category_map.values())
    lulcc_list.append(identifier_column)
    lulcc_list.append("Year")
    
    # Create an empty list to catch all the zonal statistics calculations (lists of dictionaries)
    lists_of_annual_zonal_statistics = []

    # Run a for loop to calculate zonal statistics for each year of land cover data (1992-2015)
    for i in range(24):
        lists_of_annual_zonal_statistics.append(zonal_stats(zones_statistics_boundaries,
                                                        regional_landcover_data, 
                                                        band=i+1, 
                                                        categorical=True, 
                                                        category_map=category_map))
        
    # Convert the list of lists of dictionaries to a long-form pandas DataFrame
    annual_zonal_statistics_df = pd.DataFrame(lists_of_annual_zonal_statistics).stack().apply(pd.Series)

    # Rename the DataFrame multiindex levels 
    annual_zonal_statistics_df.index.names = ['Year', identifier_column]
    
    # Reset the DataFrame to a single index ("county")
    annual_zonal_statistics_df.reset_index(level="Year", inplace=True)

    # Modify time index to reflect common era years
    annual_zonal_statistics_df["Year"] = pd.to_datetime(annual_zonal_statistics_df["Year"] + 1992, format="%Y")
    
    # Reset index of DataFrame entirely
    annual_zonal_statistics_df.reset_index(inplace=True)
    
    # Define nested function to remove duplicate elements from a list
    def removeDuplicates(listofElements):
    
    # Create an empty list to store unique elements
        uniqueList = []
    
    # Iterate over the original list and for each element add it to uniqueList, if it isn't already there
        for elem in listofElements:
            if elem not in uniqueList:
                uniqueList.append(elem)
    
    # Return the list of unique elements        
        return uniqueList
        
    # Calculate pixel count within each zonal statistic boundary
    annual_zonal_statistics_df["pixel_count"] = annual_zonal_statistics_df[identifier_column].map(pd.Series(dict(zip(range(len(removeDuplicates(annual_zonal_statistics_df.sum(axis=1)))), removeDuplicates(annual_zonal_statistics_df.sum(axis=1))))))
    
    # Map county names to integer identifier codes
    annual_zonal_statistics_df[identifier_column] = annual_zonal_statistics_df[identifier_column].map(gpd.read_file(zones_statistics_boundaries)[identifier_column].to_dict())
    
    # Convert pixel counts to percentages of total for each zone
    annual_percentage_df = pd.DataFrame(annual_zonal_statistics_df.fillna(0).drop([identifier_column,"Year"], axis=1).div(annual_zonal_statistics_df.pixel_count, axis=0)*100, index=annual_zonal_statistics_df.index)
    
    # Append the zonal boundary identifier column
    annual_percentage_df[identifier_column] = annual_zonal_statistics_df[identifier_column].copy()
    
    # Append the time identifer column
    annual_percentage_df["Year"] = annual_zonal_statistics_df["Year"].copy()
    
    # Remove pixel_count percent of total
    annual_percentage_df.drop(["pixel_count"], axis=1, inplace=True)
    
    # Reindex the DataFrame to the land cover categories + zonal boundary identifier and time, fill NaNs with 0
    annual_percentage_df = annual_percentage_df.reindex(lulcc_list, axis=1).fillna(0)

    # Set row index to identifier column
    annual_percentage_df.set_index("Year", inplace=True)
      
    return annual_percentage_df

# Create a variable to contain land cover zonal statistics time series values for each Arizona county
* Use ESA land cover categories

In [0]:
arizona_esa_lc_zonal_statistics_df = generate_esa_landcover_timeseries_zonal_statistics("arizona_esa_landcover_1992_2015.tif", 
                                                                                        "arizona_county_boundaries.geojson", 
                                                                                        "NAME")

# Return the zonal statistics DataFrame

In [0]:
arizona_esa_lc_zonal_statistics_df

# Subset the DataFrame for Pima County

In [0]:
arizona_esa_lc_zonal_statistics_df[arizona_esa_lc_zonal_statistics_df.NAME == "Pima"]

# Plot the Pima County subset

In [0]:
arizona_esa_lc_zonal_statistics_df[arizona_esa_lc_zonal_statistics_df.NAME == "Pima"].plot()

# Remove the legend for figure aesthetics

In [0]:
arizona_esa_lc_zonal_statistics_df[arizona_esa_lc_zonal_statistics_df.NAME == "Pima"].plot(legend=False)

# Re-calculate land cover change statistics with merged IPCC land cover categories

In [0]:
def generate_ipcc_landcover_timeseries_zonal_statistics(regional_landcover_data,
                                                        zones_statistics_boundaries, 
                                                        identifier_column):
    
    # ESA Land cover categories
    category_map = {0:"No data",
                    10:"Cropland, rainfed",
                    11:"Herbaceous cover",
                    12:"Tree or shrub cover",
                    20:"Cropland, irrigated or post-flooding",
                    30:"Mosaic cropland (>50%) / natural vegetation (tree, shrub, herbaceous cover) (<50%)",
                    40:"Mosaic natural vegetation (tree, shrub, herbaceous cover) (>50%) / cropland (<50%)",
                    50:"Tree cover, broadleaved, evergreen, closed to open (>15%)",
                    60:"Tree cover, broadleaved, deciduous, closed to open (>15%)",
                    61:"Tree cover, broadleaved, deciduous, closed (>40%)",
                    62:"Tree cover, broadleaved, deciduous, open (15-40%)",
                    70:"Tree cover, needleleaved, evergreen, closed to open (>15%)",
                    71:"Tree cover, needleleaved, evergreen, closed (>40%)",
                    72:"Tree cover, needleleaved, evergreen, open (15-40%)",
                    80:"Tree cover, needleleaved, deciduous, closed to open (>15%)",
                    81:"Tree cover, needleleaved, deciduous, closed (>40%)",
                    82:"Tree cover, needleleaved, deciduous, open (15-40%)",
                    90:"Tree cover, mixed leaf type (broadleaved and needleleaved)",
                    100:"Mosaic tree and shrub (>50%) / herbaceous cover (<50%)",
                    110:"Mosaic herbaceous cover (>50%) / tree and shrub (<50%)",
                    120:"Shrubland",
                    121:"Shrubland evergreen",
                    122:"Shrubland deciduous",
                    130:"Grassland",
                    140:"Lichens and mosses",
                    150:"Sparse vegetation (tree, shrub, herbaceous cover) (<15%)",
                    151:"Sparse tree (<15%)",
                    152:"Sparse shrub (<15%)",
                    153:"Sparse herbaceous cover (<15%)",
                    160:"Tree cover, flooded, fresh or brakish water",
                    170:"Tree cover, flooded, saline water",
                    180:"Shrub or herbaceous cover, flooded, fresh/saline/brakish water",
                    190:"Urban areas",
                    200:"Bare areas",
                    201:"Consolidated bare areas",
                    202:"Unconsolidated bare areas",
                    210:"Water bodies",
                    220:"Permanent snow and ice"}
    
    # List of land cover category values from dictionary
    lulcc_list = list(category_map.values())
    lulcc_list.append(identifier_column)
    lulcc_list.append("Year")
    
    # Create an empty list to catch all the zonal statistics calculations (lists of dictionaries)
    lists_of_annual_zonal_statistics = []

    # Run a for loop to calculate zonal statistics for each year of land cover data (1992-2015)
    for i in range(24):
        lists_of_annual_zonal_statistics.append(zonal_stats(zones_statistics_boundaries,
                                                        regional_landcover_data, 
                                                        band=i+1, 
                                                        categorical=True, 
                                                        category_map=category_map))
        
    # Convert the list of lists of dictionaries to a long-form pandas DataFrame
    annual_zonal_statistics_df = pd.DataFrame(lists_of_annual_zonal_statistics).stack().apply(pd.Series)

    # Rename the DataFrame multiindex levels 
    annual_zonal_statistics_df.index.names = ['Year', identifier_column]
    
    # Reset the DataFrame to a single index ("county")
    annual_zonal_statistics_df.reset_index(level="Year", inplace=True)

    # Modify time index to reflect common era years
    annual_zonal_statistics_df["Year"] = pd.to_datetime(annual_zonal_statistics_df["Year"] + 1992, format="%Y")
    
    # Reset index of DataFrame entirely
    annual_zonal_statistics_df.reset_index(inplace=True)
    
    # Define nested function to remove duplicate elements from a list
    def removeDuplicates(listofElements):
    
    # Create an empty list to store unique elements
        uniqueList = []
    
    # Iterate over the original list and for each element add it to uniqueList, if it isn't already there
        for elem in listofElements:
            if elem not in uniqueList:
                uniqueList.append(elem)
    
    # Return the list of unique elements        
        return uniqueList
        
    # Calculate pixel count within each zonal statistic boundary
    annual_zonal_statistics_df["pixel_count"] = annual_zonal_statistics_df[identifier_column].map(pd.Series(dict(zip(range(len(removeDuplicates(annual_zonal_statistics_df.sum(axis=1)))), removeDuplicates(annual_zonal_statistics_df.sum(axis=1))))))
    
    # Map county names to integer identifier codes
    annual_zonal_statistics_df[identifier_column] = annual_zonal_statistics_df[identifier_column].map(gpd.read_file(zones_statistics_boundaries)[identifier_column].to_dict())
    
    # Convert pixel counts to percentages of total for each zone
    annual_percentage_df = pd.DataFrame(annual_zonal_statistics_df.fillna(0).drop([identifier_column,"Year"], axis=1).div(annual_zonal_statistics_df.pixel_count, axis=0)*100, index=annual_zonal_statistics_df.index)
    
    # Append the zonal boundary identifier column
    annual_percentage_df[identifier_column] = annual_zonal_statistics_df[identifier_column].copy()
    
    # Append the time identifer column
    annual_percentage_df["Year"] = annual_zonal_statistics_df["Year"].copy()
    
    # Remove pixel_count percent of total
    annual_percentage_df.drop(["pixel_count"], axis=1, inplace=True)
    
    # Reindex the DataFrame to the land cover categories + zonal boundary identifier and time, fill NaNs with 0
    annual_percentage_df = annual_percentage_df.reindex(lulcc_list, axis=1).fillna(0)
      
    # Create empty DataFrame to populate with annual ESA land cover values aggregated to IPCC land cover groupings
    cleaned_df = pd.DataFrame(index=annual_percentage_df.index)
        
    # IPCC "Agriculture" category merged from ESA CCI Land Cover classes
    cleaned_df["Agriculture"] = annual_percentage_df["Cropland, rainfed"].values + annual_percentage_df["Herbaceous cover"].values + annual_percentage_df["Tree or shrub cover"].values + annual_percentage_df['Cropland, irrigated or post-flooding'].values + annual_percentage_df["Mosaic cropland (>50%) / natural vegetation (tree, shrub, herbaceous cover) (<50%)"].values + annual_percentage_df["Mosaic natural vegetation (tree, shrub, herbaceous cover) (>50%) / cropland (<50%)"].values
    
    # IPCC "Forest" category merged from ESA CCI Land Cover classes
    cleaned_df["Forest"] = annual_percentage_df['Tree cover, broadleaved, evergreen, closed to open (>15%)'].values + annual_percentage_df['Tree cover, broadleaved, deciduous, closed to open (>15%)'].values + annual_percentage_df['Tree cover, broadleaved, deciduous, closed (>40%)'].values + annual_percentage_df['Tree cover, broadleaved, deciduous, open (15-40%)'].values + annual_percentage_df['Tree cover, needleleaved, evergreen, closed to open (>15%)'].values + annual_percentage_df['Tree cover, needleleaved, evergreen, closed (>40%)'].values + annual_percentage_df['Tree cover, needleleaved, evergreen, open (15-40%)'].values + annual_percentage_df['Tree cover, needleleaved, deciduous, closed to open (>15%)'].values + annual_percentage_df['Tree cover, needleleaved, deciduous, closed (>40%)'].values + annual_percentage_df['Tree cover, needleleaved, deciduous, open (15-40%)'].values + annual_percentage_df['Tree cover, mixed leaf type (broadleaved and needleleaved)'].values + annual_percentage_df['Mosaic tree and shrub (>50%) / herbaceous cover (<50%)'].values + annual_percentage_df['Tree cover, flooded, fresh or brakish water'].values + annual_percentage_df['Tree cover, flooded, saline water'].values
        
    # IPCC "Grassland" category merged from ESA CCI Land Cover classes
    cleaned_df["Grassland"] = annual_percentage_df['Mosaic herbaceous cover (>50%) / tree and shrub (<50%)'].values + annual_percentage_df['Grassland'].values
        
    # IPCC "Wetland" category merged from ESA CCI Land Cover classes
    cleaned_df["Wetland"] = annual_percentage_df['Shrub or herbaceous cover, flooded, fresh/saline/brakish water'].values
    
    # IPCC "Settlement" category merged from ESA CCI Land Cover classes
    cleaned_df["Settlement"] = annual_percentage_df['Urban areas'].values
    
    # IPCC "Other" category merged from ESA CCI Land Cover classes
    cleaned_df["Other"] = annual_percentage_df['Shrubland'].values + annual_percentage_df['Shrubland evergreen'].values + annual_percentage_df['Shrubland deciduous'].values + annual_percentage_df['Lichens and mosses'].values + annual_percentage_df['Sparse vegetation (tree, shrub, herbaceous cover) (<15%)'].values + annual_percentage_df['Sparse tree (<15%)'].values + annual_percentage_df['Sparse shrub (<15%)'].values + annual_percentage_df['Sparse herbaceous cover (<15%)'].values + annual_percentage_df['Bare areas'].values + annual_percentage_df['Consolidated bare areas'].values + annual_percentage_df['Unconsolidated bare areas'].values + annual_percentage_df['Water bodies'].values
        
    # "No data" category
    cleaned_df['No data'] = annual_percentage_df['No data'].values
    
    # Zonal statistics boundary identifier
    cleaned_df[identifier_column] = annual_percentage_df[identifier_column].copy()

    # Zonal statistics date identifier
    cleaned_df["Year"] = annual_percentage_df["Year"].copy()

    # Index on identifier_column and "Year"
    cleaned_df.set_index("Year", inplace=True)
  
    return cleaned_df

# Create variable for IPCC land cover category results

In [0]:
arizona_ipcc_lc_zonal_statistics_df = generate_ipcc_landcover_timeseries_zonal_statistics("arizona_esa_landcover_1992_2015.tif", 
                                                                                          "arizona_county_boundaries.geojson", 
                                                                                          "NAME")

# Return IPCC results

In [0]:
arizona_ipcc_lc_zonal_statistics_df

# Subset IPCC results for Pima County

In [0]:
arizona_ipcc_lc_zonal_statistics_df[arizona_ipcc_lc_zonal_statistics_df.NAME == "Pima"]

# Plot the IPCC Pima County subset

In [0]:
arizona_ipcc_lc_zonal_statistics_df[arizona_ipcc_lc_zonal_statistics_df.NAME == "Pima"].plot()

# Write a convenience function to visualize land cover change statistics and the zonal boundaries

In [0]:
def make_a_cool_visualization(boundary_vector, land_cover_zonal_statistics_time_series_df, attribute_join_column, zone_of_interest):
  gdf = gpd.read_file(boundary_vector)
  df = land_cover_zonal_statistics_time_series_df[land_cover_zonal_statistics_time_series_df[attribute_join_column] == zone_of_interest]
  df = df.drop([attribute_join_column], axis=1)

  fig, ax = plt.subplots(1,2)
  gdf.plot(ax=ax[0], cmap="Greys", zorder=1)
  gdf[gdf[attribute_join_column] == zone_of_interest].plot(ax=ax[0], color="Orange", zorder=3)  
  df.plot(ax=ax[1], cmap="Accent")
  ax[1].legend(loc='upper center', bbox_to_anchor=(-0.1, -0.2), fancybox=True, shadow=True, ncol=3)
  plt.suptitle("Land Cover Change in {}".format(zone_of_interest), fontweight="bold")
  plt.plot()

# Call the function for Pima County

In [0]:
make_a_cool_visualization("arizona_county_boundaries.geojson",
                          arizona_ipcc_lc_zonal_statistics_df,
                          "NAME",
                          "Pima")

# Call the function Maricopa County

In [0]:
make_a_cool_visualization("arizona_county_boundaries.geojson",
                          arizona_ipcc_lc_zonal_statistics_df,
                          "NAME",
                          "Maricopa")

# Generate a list of counties in Arizona from the list of unique zones in the land cover statistics data set

In [0]:
arizona_ipcc_lc_zonal_statistics_df.NAME.unique().tolist()

# Iterate through all counties in Arizona with a for loop

In [0]:
for county in arizona_ipcc_lc_zonal_statistics_df.NAME.unique().tolist():
  make_a_cool_visualization("arizona_county_boundaries.geojson",
                          arizona_ipcc_lc_zonal_statistics_df,
                          "NAME",
                          county)

# Thanks!